In [ ]:
import requests
import json
import time

def delete_empty_tags(api_url, api_key):
    headers = {
        'Authorization': api_key
    }
    
    # Get all tags
    tag_list_url = f"{api_url}/api/action/tag_list"
    response = requests.get(tag_list_url)
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f"Error retrieving tags. Status code: {response.status_code}")
        print(f"Response content: {response.text}")
        return
    
    # Try to parse the JSON response
    try:
        tags_data = response.json()
        if 'result' not in tags_data:
            print("Error: 'result' key not found in response")
            print(f"Response content: {response.text}")
            return
        
        tags = tags_data['result']
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response: {e}")
        print(f"Response content: {response.text[:200]}...")  # Print first 200 chars
        return
    
    delete_count = 0
    
    for tag in tags:
        # Check if the tag has associated datasets
        package_search_url = f"{api_url}/api/action/package_search"
        search_response = requests.get(package_search_url, params={'fq': f'tags:{tag}'})
        
        # Check if search response is valid
        try:
            search_result = search_response.json()
            if 'result' not in search_result or 'count' not in search_result['result']:
                print(f"Error: Invalid search response format for tag '{tag}'")
                continue
                
            if search_result['result']['count'] == 0:
                # Delete tag if it has no datasets
                delete_url = f"{api_url}/api/action/tag_delete"
                delete_response = requests.post(
                    delete_url,
                    json={'id': tag},
                    headers=headers
                )
                
                if delete_response.status_code == 200:
                    delete_count += 1
                    print(f"Deleted tag: {tag}")
                else:
                    print(f"Failed to delete tag: {tag}. Status code: {delete_response.status_code}")
                    print(f"Response: {delete_response.text[:100]}...")
                
                # Add a small delay to avoid overwhelming the API
                time.sleep(0.5)
                
        except json.JSONDecodeError as e:
            print(f"Error parsing search results for tag '{tag}': {e}")
            continue
    
    print(f"\nTotal empty tags deleted: {delete_count}")

# Usage example:
if __name__ == "__main__":
    api_url = ""
    api_key = ""
    delete_empty_tags(api_url, api_key)

Deleted tag: economy
Deleted tag: Local Government

Total empty tags deleted: 2
